In [1]:
import xarray as xr
import distributed


In [44]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

c:\Users\haris\miniconda3\envs\torch\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54717 instead
  warnings.warn(
2024-06-10 15:36:36,138 - distributed.deploy.spec - WARNING - Cluster closed without starting up


RuntimeError: Cluster failed to start: [Errno 2] No such file or directory: 'C:\\Users\\haris\\miniconda3\\envs\\torch\\Lib\\site-packages\\bokeh\\server\\views\\bokeh.ico'

# Load IFS data

In [2]:
ds_ifs_pl = xr.open_dataset(r'D:\Projects\NIOT\v1\ensemble_transformer\data\pl.nc',chunks={'time': 2})
ds_ifs_sfc = xr.open_dataset(r'D:\Projects\NIOT\v1\ensemble_transformer\data\sfc.nc', chunks={'time': 2})

In [4]:
ds_ifs_gh = ds_ifs_pl['gh'].to_dataset('level')
ds_ifs_gh = ds_ifs_gh.rename({500: 'gh_500', 850: 'gh_850'})

In [5]:
ds_ifs_t = ds_ifs_pl['t'].to_dataset("level")
ds_ifs_t = ds_ifs_t.rename({500: 't_500', 850: 't_850'})

In [6]:
ds_ifs_merged = xr.merge([ds_ifs_sfc, ds_ifs_t, ds_ifs_gh])

In [7]:
ds_ifs_merged = ds_ifs_merged.isel(latitude=slice(None, None, -1))
ds_ifs_merged = ds_ifs_merged.rename({'number': 'ensemble'})

In [8]:
ds_ifs_train = ds_ifs_merged.sel()
ds_ifs_train.to_zarr(
    '../data/processed/ifs/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

In [9]:
ds_ifs_test = ds_ifs_merged.sel()
ds_ifs_test.to_zarr(
    '../data/processed/ifs/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)